In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time

In [40]:
def get_QA_content(id_number):
    url = "https://investoredu.twse.com.tw/pages/TWSE_InvestmentQA.aspx?ID=15"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    base = "ContentPlaceHolder1_"
    ids = {
        "question": f"{base}Label1_{id_number}",
        "answer": f"{base}Label2_{id_number}",
        "url_label": f"{base}lblURL1_{id_number}",
        "url": f"{base}hplURL1_{id_number}",
    }

    def safe_get(id_, is_url=False):
        tag = soup.find(id=id_)
        if not tag:
            return ""
        return tag["href"].strip() if is_url else tag.get_text(strip=True)

    question = safe_get(ids["question"])
    answer_main = safe_get(ids["answer"])
    url_label = safe_get(ids["url_label"])
    url = safe_get(ids["url"], is_url=True)

    answer_parts = [answer_main]
    if url_label:
        answer_parts.append(url_label)
    if url:
        answer_parts.append(url)

    final_answer = "\n".join(answer_parts).strip()

    return {
        "問題": question,
        "答案": final_answer
    }

all_qa = []
for i in range(11):
    try:
        data = get_QA_content(i)
        if data["問題"] or data["答案"]:
            all_qa.append(data)
            print(f"成功存取第 {i + 1} 筆")
            print(data["問題"])
            print("------------------------")
            print(data["答案"])
        else:
            print(f"第 {i} 筆無資料")
    except Exception as e:
        print(f"第{i} 筆錯誤: {e}")
    time.sleep(1)

df_original = pd.read_csv("twse.csv")
# 轉為 DataFrame
df_new = pd.DataFrame(all_qa)

# 合併舊 + 新資料
df_merged = pd.concat([df_original, df_new], ignore_index=True)

# 存回原本的檔案（或另存新檔）
df_merged.to_csv("twse.csv", index=False, encoding="utf-8-sig")

    

成功存取第 1 筆
何謂證券業務借貸款項？
------------------------
指證券商與客戶約定，為因應客戶購買有價證券或其他商品之需，所從事之資金融通業務。證券商辦理客戶借貸款項業務，應訂定借貸款項契約書，依契約書約定事項執行，以明雙方權利及義務。
成功存取第 2 筆
T+5型借貸款項與半年型借貸款項之區別？
------------------------
T+5型係以客戶買進之證券為擔保，借貸期間為成交日次二至次五營業日(T+2~T+5)，客戶應於成交日次二營業日上午11時前提出申請。半年型借貸款項客戶可以其持有之有價證券為擔保，借貸期限不超過6個月，證券商並得視客戶信用狀況展延二次，故最長為18個月，客戶應於買進證券成交日次一營業日上午12時前提出申請。
成功存取第 3 筆
客戶申請證券業務借貸款項可融通之金額如何計算？
------------------------
融通金額以每一客戶成交日買進、賣出證券價金相抵後之應付價金、相關手續費及稅負為限。此外，證券商仍應依徵信程序核定客戶得融通額度，且客戶應提供其所申請總授信額度百分之三十以上之財產證明。
成功存取第 4 筆
華僑及外國人是否可以申請證券業務借貸款項及不限用途款項借貸？
------------------------
境內外華僑及外國人均可申請證券業務借貸款項；不限用途款項借貸則僅開放境內之華僑及外國人申請。
成功存取第 5 筆
投資人是否可以賣出其所買進證券方式償還融通款項？
------------------------
可以，且若投資人係以其買進證券為擔保者，於融通期限屆滿前賣出者，均應優先償還其借貸款項。
成功存取第 6 筆
融通期限屆滿未清償之處理？
------------------------
證券商應依操作辦法第27條規定，處分客戶該筆借貸之擔保品及抵繳之有價證券或其他商品，有不足者應通知客戶次一營業日補足，未補足者證券商得於債務清償必要範圍內處分客戶提供之其他擔保品，尚不足部分通知客戶限期清償，未結清者即為違規。
成功存取第 7 筆
何謂不限用途款項借貸？
------------------------
指證券商與客戶約定，以客戶持有之有價證券或其他商品為擔保，所從事之資金融通業務，融通期限以6個月為限，證券商並得視客戶信用狀況展延二次。
成功存取第 8

In [42]:
# 展示抓下來的答案
data = get_QA_content(0)
print(data["答案"])

指證券商與客戶約定，為因應客戶購買有價證券或其他商品之需，所從事之資金融通業務。證券商辦理客戶借貸款項業務，應訂定借貸款項契約書，依契約書約定事項執行，以明雙方權利及義務。


In [41]:
import pandas as pd

#合併先前存取的內容
files = ["twse.csv", "stock.csv"]

dfs = [pd.read_csv(f)[["問題", "答案"]] for f in files]
merged_df = pd.concat(dfs, ignore_index=True)
merged_df.to_csv("twse.csv", index=False, encoding="utf-8-sig")
